In [1]:
import sys
sys.path.insert(0,'/home/csunix/schtmt/NewFolder/caffe_Sep_anaconda/python')
import numpy as np
import matplotlib.pyplot as plt
import caffe
import h5py
import cv2
import os
import time
caffe.set_device(0)
caffe.set_mode_gpu()
# caffe.set_mode_cpu()

net = caffe.Net('test2_ped1.prototxt',
                '/usr/not-backed-up/MODELS_DATA/models/ped1_iter_360000.caffemodel', 
                caffe.TEST) #570000
# print([(k, v.data.shape) for k, v in net.blobs.items()]) # Python data type: list

In [2]:
import fnmatch
import os
from PIL import Image
frame_path='/usr/not-backed-up/1_DATABASE/UCSD_Anomaly_Dataset.tar/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/'
save_path = '/usr/not-backed-up/1_convlstm/Ped1_prediction6/pixel_error'
if not os.path.exists(save_path):
    os.mkdir(save_path)
total_time = 0
total_frame = 0
for seq in range(7,36):#(36)
    print(seq)    
    # load images
    if seq < 9:                
        dirpath = (frame_path + 'Test00%d' % (seq+1))
    else:
        dirpath = (frame_path + 'Test0%d' % (seq+1))
    nfr = len(fnmatch.filter(os.listdir(dirpath), '*.tif'))
#         frame_seq = np.zeros((227,227,nfr),dtype=np.float64)
#     start_time = time.clock()
    pixel_error = np.zeros([5,227,227,nfr-9])
    for fr in range(nfr-9):     
#         print(fr)
        if fr == 0:
            frame_seq = np.zeros((10,227,227))
            for idx in range(10):
                if (fr+1+idx) < 10:
                    fr_id = '00%d' % (fr+1+idx)                                       
                elif (fr+1+idx) < 100:
                    fr_id = '0%d' % (fr+1+idx)                
                else:
                    fr_id = '%d' % (fr+1+idx) 
                im1 = Image.open(dirpath + '/' + (fr_id) + '.tif')
                im1 = np.array(im1.resize((227,227),Image.BILINEAR))#size [width,height]            
                im1 = np.reshape(im1,[1,227,227])
                im1 = im1.astype(float) / 255.
                frame_seq[idx,:,:] = im1        
        else:
            if (fr+10) < 10:
                fr_id = '00%d' % (fr+10)                                       
            elif (fr+10) < 100:
                fr_id = '0%d' % (fr+10)                
            else:
                fr_id = '%d' % (fr+10) 
            im1 = Image.open(dirpath + '/' + (fr_id) + '.tif')
            im1 = np.array(im1.resize((227,227),Image.BILINEAR))#size [width,height]            
            im1 = np.reshape(im1,[1,227,227])
            im1 = im1.astype(float) / 255.
            
            frame_seq = np.append(frame_seq[1:10,:,:],im1,0)   
    

#         cur_vol = frame_seq[fr:fr+10,:,:]
#         cur_vol = cur_vol.reshape([1,10,1,227,227])
        net.blobs['input'].data[...] = np.reshape(frame_seq[0:5,:,:],[1,5,1,227,227])
        out = net.forward()        
        prediction = net.blobs['deconv3'].data
        prediction = np.reshape(prediction,[5,227,227])
        err = np.power((frame_seq[5:10,:,:] - prediction),2)
#         print(err.shape) 
        pixel_error[:,:,:,fr] = err 
    filename = ("test_%d_error.h5" % (seq+1))        

    h5f = h5py.File(os.path.join(save_path,filename), 'w')
    h5f.create_dataset('pixel_error', data=pixel_error)                
    h5f.close()   
# print(total_time/total_frame, "second")        
# print("Processing time:", total_time/total_frame, "second")        

7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [ ]:
plt.figure()
plt.plot(np.arange(fr+1).reshape(1,fr+1),frame_error,'r*')
plt.show()